# Engineering the Fiscal Fortitude Quotient

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("whitegrid")
plt.rcParams["lines.linewidth"] = 1
plt.rcParams["axes.edgecolor"] = "k"

In [3]:
df = pd.read_csv("data/base/economic_data_weekly.csv", index_col="DATE", parse_dates=True)
df.head()

,GDP,CPIAUCNS,UNRATE,FEDFUNDS,DTWEXM,T10Y2Y,M2,PCE,HOANBS,BUSINV,...,WM2NS,UMCSENT,OILPRICE,BAMLH0A0HYM2,GS10,CP,STLFSI,USSLIND,GDPC1,USROA
DATE,,,,,,,,,,,,,,,,,,,,,
1997-01-03,8362.655000,159.1,5.30,5.250,88.83620,0.550,3820.0,5411.10,98.153000,1007785.0,...,3836.3,97.40,25.170,3.093333,6.580,546.764000,0.728,1.610,11291.665000,1.270000
1997-01-10,8374.668077,159.2,5.28,5.238,89.02160,0.580,3830.4,5415.68,98.189538,1008371.2,...,3874.1,97.86,24.578,3.112000,6.548,548.187077,0.774,1.652,11306.100769,1.273077
1997-01-17,8386.681154,159.3,5.26,5.226,89.46776,0.558,3833.9,5420.26,98.226077,1008957.4,...,3861.2,98.32,23.986,3.052000,6.516,549.610154,0.732,1.694,11320.536538,1.276154
1997-01-24,8398.694231,159.4,5.24,5.214,90.67085,0.550,3834.9,5424.84,98.262615,1009543.6,...,3841.2,98.78,23.394,2.996000,6.484,551.033231,0.715,1.736,11334.972308,1.279231
1997-01-31,8410.707308,159.5,5.22,5.202,91.54040,0.592,3836.7,5429.42,98.299154,1010129.8,...,3804.4,99.24,22.802,2.954000,6.452,552.456308,0.747,1.778,11349.408077,1.282308


## The Fiscal Fortitude Quotient (FFQ)

The FFQ uses several economic indicators:
1. GDP
2. Unemployment Rate
3. Inflation
4. Federal Funds Rate
5. Oil Prices

Each is processed through its own piecewise function to capture complex relationships. Below are the functions and their explanations.

### 1. GDP Component

$$
f(x) = 
\begin{cases}
    x & \text{if } x < 0.8 \\
    0.8 + 0.2(1 - (x - 0.8)^2) & \text{if } x \geq 0.8
\end{cases}
$$

In [6]:
def gdp_ffq(x):
    return x if x < 0.8 else 0.8 + 0.2 * (1 - (x - 0.8)**2)

**Rationale**: Moderate GDP growth is beneficial. However, extremely high GDP growth may indicate an overheated economy or a bubble. Hence, the function plateaus and then penalizes values above 0.8.

### 2. Unemployment Rate Component

$$
f_{\text{UNRATE}}(x) = 
\begin{cases}
    x & \text{if } x > 0.2 \\
    0.2 - 0.2(1 - (x - 0.2)^2) & \text{if } x \leq 0.2
\end{cases}
$$

In [7]:
def unrate_ffq(x):
    return x if x > 0.2 else 0.2 - 0.2 * (1 - (x - 0.2)**2)

**Rationale**: Low unemployment is generally good, but extremely low levels may indicate labor shortages. Therefore, the function penalizes low values below 0.2.

### 3. Inflation Component

$$
f_{\text{CPIAUCNS}}(x) = 1 - |x - 0.5| \times 2
$$

In [11]:
def inflation_ffq(x):
    return 1 - 2 * np.abs(x - 0.5)

**Rationale**: Moderate inflation is ideal. Both high inflation and deflation are unfavorable for economic health. The function is symmetrical around 0.5, penalizing both high and low values.

### 4. Federal Funds Rate Component

$$
f_{\text{FEDFUNDS}}(x) =
\begin{cases}
    x & \text{if } x > 0.2 \\
    0.2 - 0.2(1 - (x - 0.2)^2) & \text{if } x \leq 0.2
\end{cases}
$$

In [12]:
def fedfunds_ffq(x):
    return x if x > 0.2 else 0.2 - 0.2 * (1 - (x - 0.2)**2)

**Rationale**: Low federal funds rates are generally favorable for stimulating economic activity. However, extremely low rates can indicate economic stress. Thus, the function penalizes values below 0.2.

### 5. Oil Price Component

$$
f_{\text{OILPRICE}}(x) = 1 - |x - 0.5| \times 2
$$

In [13]:
def oil_ffq(x):
    return 1 - 2 * np.abs(x - 0.5)

**Rationale**: Moderate oil prices are beneficial for both producers and consumers. Extremely high or low prices can cause economic stress, and thus the function penalizes both.

In [35]:
# Define the function to compute the Financial Fortitude Quotient (FFQ)
def compute_ffq(transformations, ffq_components_normalized_df):
    
    # Create a copy of the DataFrame to avoid modifying the original
    ffq_components_normalized_df = ffq_components_normalized_df.copy()
    
    # Get the column names, which represent the economic components
    components = ffq_components_normalized_df.columns
    
    # Count the number of economic components
    n_components = len(components)
    
    # Apply the respective piecewise function to each economic component
    for component in components:
        ffq_components_normalized_df[component] = \
            ffq_components_normalized_df[component].apply(transformations[component])
    
    # Sum up the transformed components and normalize by the number of components
    return ffq_components_normalized_df.sum(axis=1) / n_components

# Define a dictionary that maps each economic component to its piecewise function
transformations = {
    'GDP': gdp_ffq,               # Function for Gross Domestic Product
    'UNRATE': unrate_ffq,         # Function for Unemployment Rate
    'CPIAUCNS': inflation_ffq,    # Function for Inflation
    'FEDFUNDS': fedfunds_ffq,     # Function for Federal Funds Rate
    'OILPRICE': oil_ffq           # Function for Oil Price
}

In [32]:
ffq_components_df = df[transformations.keys()]
ffq_components_normalized_df = (ffq_components_df - ffq_components_df.min()) / \
                               (ffq_components_df.max() - ffq_components_df.min())

df_with_ffq = df.copy()
df_with_ffq["FFQ"] = compute_ffq(transformations, ffq_components_normalized_df)

df_with_ffq.head()

,GDP,CPIAUCNS,UNRATE,FEDFUNDS,DTWEXM,T10Y2Y,M2,PCE,HOANBS,BUSINV,...,UMCSENT,OILPRICE,BAMLH0A0HYM2,GS10,CP,STLFSI,USSLIND,GDPC1,USROA,FFQ
DATE,,,,,,,,,,,,,,,,,,,,,
1997-01-03,8362.655000,159.1,5.30,5.250,88.83620,0.550,3820.0,5411.10,98.153000,1007785.0,...,97.40,25.170,3.093333,6.580,546.764000,0.728,1.610,11291.665000,1.270000,0.205587
1997-01-10,8374.668077,159.2,5.28,5.238,89.02160,0.580,3830.4,5415.68,98.189538,1008371.2,...,97.86,24.578,3.112000,6.548,548.187077,0.774,1.652,11306.100769,1.273077,0.203694
1997-01-17,8386.681154,159.3,5.26,5.226,89.46776,0.558,3833.9,5420.26,98.226077,1008957.4,...,98.32,23.986,3.052000,6.516,549.610154,0.732,1.694,11320.536538,1.276154,0.201802
1997-01-24,8398.694231,159.4,5.24,5.214,90.67085,0.550,3834.9,5424.84,98.262615,1009543.6,...,98.78,23.394,2.996000,6.484,551.033231,0.715,1.736,11334.972308,1.279231,0.199910
1997-01-31,8410.707308,159.5,5.22,5.202,91.54040,0.592,3836.7,5429.42,98.299154,1010129.8,...,99.24,22.802,2.954000,6.452,552.456308,0.747,1.778,11349.408077,1.282308,0.198018


In [36]:
df_with_ffq.to_csv("data/main/economic_data_with_ffq.csv")